In [1]:
import psycopg2
import os

In [2]:
postgres_uri = os.environ.get('URI')

In [3]:
def execute_sql(sql):
    try:
        conn = psycopg2.connect(postgres_uri)
        cur = conn.cursor()
        cur.execute(sql)
        conn.commit()
    finally:
        if conn is not None:
            conn.close()

In [4]:
def execute_sql_results(sql):
    try:
        conn = psycopg2.connect(postgres_uri)
        cur = conn.cursor()
        cur.execute(sql)
        rows = cur.fetchall()
        return rows
    finally:
        if conn is not None:
            conn.close()

In [5]:
sql_create_retriever = """
        select aidb.create_retriever_for_table(
        	name => 'film_retriever',
        	model_name => 'bert',
        	source_table => 'film',
            source_data_column => 'description',
        	source_data_type => 'Text',
        	source_key_column => 'film_id',
        	distance_operator => 'Cosine'
        )
    """

sql_prime_embeddings = "select aidb.bulk_embedding('film_retriever')"

sql_enable_auto_embedding = "select aidb.enable_auto_embedding_for_table('film_retriever')"

In [6]:
execute_sql(sql_create_retriever)
execute_sql(sql_prime_embeddings)
execute_sql(sql_enable_auto_embedding)

In [7]:
sql_find_related_film_titles = """
        select f.film_id, f.title, f.description, rk.distance
        from aidb.retrieve_key('film_retriever', 'A movie about cats in Australia', 10) rk
        join film f on rk.key::INTEGER = f.film_id
        order by 4 asc
    """

rows = execute_sql_results(sql_find_related_film_titles)
for row in rows:
    print(row)

(592, 'Monster Spartacus', 'A Fast-Paced Story of a Waitress And a Cat who must Fight a Girl in Australia', 0.30614852905265544)
(384, 'Grosse Wonderful', 'A Epic Drama of a Cat And a Explorer who must Redeem a Moose in Australia', 0.329498132526978)
(298, 'Eyes Driving', 'A Thrilling Story of a Cat And a Waitress who must Fight a Explorer in The Outback', 0.38572982209438056)
(339, 'Frogmen Breaking', 'A Unbelieveable Yarn of a Mad Scientist And a Cat who must Chase a Lumberjack in Australia', 0.4134015933753584)
(157, 'Clockwork Paradise', 'A Insightful Documentary of a Technical Writer And a Feminist who must Challenge a Cat in A Baloon', 0.44302881443147135)
(176, 'Congeniality Quest', 'A Touching Documentary of a Cat And a Pastry Chef who must Find a Lumberjack in A Baloon', 0.4532806706240152)
(248, 'Dozen Lion', 'A Taut Drama of a Cat And a Girl who must Defeat a Frisbee in The Canadian Rockies', 0.462680426995733)
(280, 'Empire Malkovich', 'A Amazing Story of a Feminist And a C